# GNOD song sugester


## Web Scraping Hot 100 from Billboard

In [1]:
# 100 songs and their respective artists from https://www.billboard.com/charts/hot-100

In [2]:
# importing libraries

from bs4 import BeautifulSoup
import re 
import requests
import pandas as pd
from tqdm.notebook import tqdm
import random
from random import randint
import warnings
warnings.filterwarnings("ignore")

In [3]:
# storing link in a variable
url = "https://www.billboard.com/charts/hot-100"
# downloading  html with a get request
response = requests.get(url)
# check response status code 
response.status_code

200

In [4]:
# parsing and storing the contents of the url call
billboardsoup = BeautifulSoup(response.content, 'html.parser')

In [5]:
# looping the songs
top_100 = len(billboardsoup.select('h3.c-title.a-no-trucate'))
song = []
artist = []

for i in tqdm(range(top_100)):
    song.append(billboardsoup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
    artist.append(billboardsoup.select('span.c-label.a-font-primary-s')[i].get_text(strip=True))

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
billboard_top100 = pd.DataFrame({'song':song,'artist':artist})

In [7]:
# more libraries, you can never have enough

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint

In [8]:
client_id = str(getpass.getpass('client_id: '))
client_secret = str(getpass.getpass('client_secret: '))
# embedding getpass credentials in the access key
sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret))

client_id: ········
client_secret: ········


In [9]:
first_playlist = sp.user_playlist_tracks("solitude collective", "1IEQ8C3G1qT0W80muYgROT")

In [10]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
first_results = get_tracks("solitude collective", "1IEQ8C3G1qT0W80muYgROT")

In [11]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = first_results
for r in first_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [12]:

features_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:]))
features = pd.concat([features_info, features_info2, features_info3, features_info4, features_info5])
features.reset_index(drop=True, inplace=True)

In [13]:
first_playlist = pd.concat([playlist_info, features], axis=1)
first_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Carry on Wayward Son,2SUpC3UgKwLVOS2FtZif9N,Kansas,0.452,0.788,7,-5.712,1,0.0326,...,0.4410,0.7930,126.739,audio_features,2SUpC3UgKwLVOS2FtZif9N,spotify:track:2SUpC3UgKwLVOS2FtZif9N,https://api.spotify.com/v1/tracks/2SUpC3UgKwLV...,https://api.spotify.com/v1/audio-analysis/2SUp...,322093,4
1,1,Jump,2PfBSkCKZDXMsJogMT0wML,Van Halen,0.547,0.822,0,-7.201,1,0.0292,...,0.0645,0.7740,130.138,audio_features,2PfBSkCKZDXMsJogMT0wML,spotify:track:2PfBSkCKZDXMsJogMT0wML,https://api.spotify.com/v1/tracks/2PfBSkCKZDXM...,https://api.spotify.com/v1/audio-analysis/2PfB...,239560,4
2,2,Rock You Like A Hurricane,0XIvZ82aDF7JiSi3ZE320u,Scorpions,0.482,0.601,7,-12.846,1,0.0486,...,0.2490,0.7740,125.619,audio_features,0XIvZ82aDF7JiSi3ZE320u,spotify:track:0XIvZ82aDF7JiSi3ZE320u,https://api.spotify.com/v1/tracks/0XIvZ82aDF7J...,https://api.spotify.com/v1/audio-analysis/0XIv...,252493,4
3,3,Back In Black,08mG3Y1vljYA6bvDt4Wqkj,AC/DC,0.310,0.700,9,-5.678,1,0.0470,...,0.0828,0.7630,188.386,audio_features,08mG3Y1vljYA6bvDt4Wqkj,spotify:track:08mG3Y1vljYA6bvDt4Wqkj,https://api.spotify.com/v1/tracks/08mG3Y1vljYA...,https://api.spotify.com/v1/audio-analysis/08mG...,255493,4
4,4,Hot Blooded,6xMHglHoafdDFGXS6qfwSH,Foreigner,0.705,0.806,0,-4.830,1,0.0814,...,0.0676,0.7630,117.945,audio_features,6xMHglHoafdDFGXS6qfwSH,spotify:track:6xMHglHoafdDFGXS6qfwSH,https://api.spotify.com/v1/tracks/6xMHglHoafdD...,https://api.spotify.com/v1/audio-analysis/6xMH...,268693,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,423,I'd Be Lost If It Wasn't for You,64FhrwziLZYXXTtmhgk9Xh,Elijah Honey,0.639,0.743,5,-7.198,1,0.0527,...,0.3950,0.8490,120.415,audio_features,64FhrwziLZYXXTtmhgk9Xh,spotify:track:64FhrwziLZYXXTtmhgk9Xh,https://api.spotify.com/v1/tracks/64FhrwziLZYX...,https://api.spotify.com/v1/audio-analysis/64Fh...,181893,4
424,424,My Curse,6zKF4293k44ItKWJJgrhXv,Killswitch Engage,0.197,0.956,10,-2.908,0,0.0663,...,0.2610,0.2090,177.018,audio_features,6zKF4293k44ItKWJJgrhXv,spotify:track:6zKF4293k44ItKWJJgrhXv,https://api.spotify.com/v1/tracks/6zKF4293k44I...,https://api.spotify.com/v1/audio-analysis/6zKF...,244773,3
425,425,Livin' On A Prayer,7qQnBfwXrw2tZNFG4Uf57N,Bon Jovi,0.520,0.887,0,-3.296,1,0.0349,...,0.1860,0.7930,122.528,audio_features,7qQnBfwXrw2tZNFG4Uf57N,spotify:track:7qQnBfwXrw2tZNFG4Uf57N,https://api.spotify.com/v1/tracks/7qQnBfwXrw2t...,https://api.spotify.com/v1/audio-analysis/7qQn...,250627,4
426,426,"Touch-A, Touch-A, Touch Me",2SK1kGbpABI9EyUbADSP5z,Susan Sarandon,0.355,0.351,1,-13.572,1,0.0621,...,0.1090,0.4310,171.896,audio_features,2SK1kGbpABI9EyUbADSP5z,spotify:track:2SK1kGbpABI9EyUbADSP5z,https://api.spotify.com/v1/tracks/2SK1kGbpABI9...,https://api.spotify.com/v1/audio-analysis/2SK1...,151922,4


In [14]:
second_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")

In [15]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
second_results = get_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")

In [16]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = second_results
for r in second_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist2_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [17]:
features2_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features2_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:]))
features2 = pd.concat([features2_info, features2_info2])
features2.reset_index(drop=True, inplace=True)

In [18]:
second_playlist = pd.concat([playlist2_info, features2], axis=1)
second_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,I Wanna Dance with Somebody (Who Loves Me),2tUBqZG2AbRi7Q0BIrVrEj,Whitney Houston,0.709,0.824,1,-8.824,1,0.0453,...,0.0888,0.867,118.818,audio_features,2tUBqZG2AbRi7Q0BIrVrEj,spotify:track:2tUBqZG2AbRi7Q0BIrVrEj,https://api.spotify.com/v1/tracks/2tUBqZG2AbRi...,https://api.spotify.com/v1/audio-analysis/2tUB...,291293,4
1,1,Billie Jean,5ChkMS8OtdzJeqyybCc9R5,Michael Jackson,0.920,0.654,11,-3.051,0,0.0401,...,0.0360,0.847,117.046,audio_features,5ChkMS8OtdzJeqyybCc9R5,spotify:track:5ChkMS8OtdzJeqyybCc9R5,https://api.spotify.com/v1/tracks/5ChkMS8OtdzJ...,https://api.spotify.com/v1/audio-analysis/5Chk...,293827,4
2,2,Time After Time,1Jj6MF0xDOMA3Ut2Z368Bx,Cyndi Lauper,0.724,0.436,0,-9.321,1,0.0282,...,0.0908,0.324,130.439,audio_features,1Jj6MF0xDOMA3Ut2Z368Bx,spotify:track:1Jj6MF0xDOMA3Ut2Z368Bx,https://api.spotify.com/v1/tracks/1Jj6MF0xDOMA...,https://api.spotify.com/v1/audio-analysis/1Jj6...,243067,4
3,3,Take on Me,2WfaOiMkCvy7F5fcp2zZ8L,a-ha,0.573,0.902,6,-7.638,0,0.0540,...,0.0928,0.876,84.412,audio_features,2WfaOiMkCvy7F5fcp2zZ8L,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L,https://api.spotify.com/v1/tracks/2WfaOiMkCvy7...,https://api.spotify.com/v1/audio-analysis/2Wfa...,225280,4
4,4,"Footloose - From ""Footloose"" Soundtrack",2vz1CsL5WBsbpBcwgboTAw,Kenny Loggins,0.583,0.905,2,-6.132,1,0.0733,...,0.0593,0.646,173.982,audio_features,2vz1CsL5WBsbpBcwgboTAw,spotify:track:2vz1CsL5WBsbpBcwgboTAw,https://api.spotify.com/v1/tracks/2vz1CsL5WBsb...,https://api.spotify.com/v1/audio-analysis/2vz1...,226827,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,I'm Gonna Be (500 Miles),66S14BkJDxgkYxLl5DCqOz,The Proclaimers,0.851,0.551,4,-5.177,1,0.0396,...,0.0827,0.807,131.922,audio_features,66S14BkJDxgkYxLl5DCqOz,spotify:track:66S14BkJDxgkYxLl5DCqOz,https://api.spotify.com/v1/tracks/66S14BkJDxgk...,https://api.spotify.com/v1/audio-analysis/66S1...,219467,4
146,146,Missing You,1Qrdlkgg9I4J7r3P4kZNwr,John Waite,0.552,0.552,11,-9.736,0,0.0364,...,0.0460,0.593,208.571,audio_features,1Qrdlkgg9I4J7r3P4kZNwr,spotify:track:1Qrdlkgg9I4J7r3P4kZNwr,https://api.spotify.com/v1/tracks/1Qrdlkgg9I4J...,https://api.spotify.com/v1/audio-analysis/1Qrd...,269760,4
147,147,Sexual Healing,3VZmChrnVW8JK6ano4gSED,Marvin Gaye,0.805,0.602,0,-9.700,0,0.0579,...,0.0581,0.744,94.382,audio_features,3VZmChrnVW8JK6ano4gSED,spotify:track:3VZmChrnVW8JK6ano4gSED,https://api.spotify.com/v1/tracks/3VZmChrnVW8J...,https://api.spotify.com/v1/audio-analysis/3VZm...,238600,4
148,148,Golden Brown,2AX5E86cn9n2dgioZEjirI,The Stranglers,0.562,0.383,6,-15.474,1,0.0304,...,0.1380,0.599,93.764,audio_features,2AX5E86cn9n2dgioZEjirI,spotify:track:2AX5E86cn9n2dgioZEjirI,https://api.spotify.com/v1/tracks/2AX5E86cn9n2...,https://api.spotify.com/v1/audio-analysis/2AX5...,206760,3


In [19]:
third_playlist = sp.user_playlist_tracks("Peter Nordestgaard", "0lNwm5xFEgRx4iDP2DLx3j")

In [20]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
third_results = get_tracks("Peter Nordestgaard", "0lNwm5xFEgRx4iDP2DLx3j")


In [21]:

song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = third_results
for r in third_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist3_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [22]:
features3_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features3_info1 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features3_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features3_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features3_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features3_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features3_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features3_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features3_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features3_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features3_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features3 = pd.concat([features3_info, features3_info1, features3_info2, features3_info3, features3_info4, features3_info5, features3_info6, features3_info7, features3_info8, features3_info9, features3_info10])
features3.reset_index(drop=True, inplace=True)

In [23]:
third_playlist = pd.concat([playlist3_info, features3_info], axis=1)
third_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Imagine - Remastered 2010,7pKfPomDEeI4TPT6EOYjn9,John Lennon,0.547,0.257,0.0,-12.358,1.0,0.0252,...,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867.0,4.0
1,1,A Whiter Shade Of Pale,3qitymULqEibr7yknRMKU4,Procol Harum,0.249,0.660,0.0,-6.905,1.0,0.0342,...,0.0891,0.435,149.813,audio_features,3qitymULqEibr7yknRMKU4,spotify:track:3qitymULqEibr7yknRMKU4,https://api.spotify.com/v1/tracks/3qitymULqEib...,https://api.spotify.com/v1/audio-analysis/3qit...,248947.0,4.0
2,2,My Sweet Lord,1chu8wc9zbNqTQ2ahROys1,George Harrison,0.537,0.685,1.0,-8.935,0.0,0.0333,...,0.0839,0.542,120.965,audio_features,1chu8wc9zbNqTQ2ahROys1,spotify:track:1chu8wc9zbNqTQ2ahROys1,https://api.spotify.com/v1/tracks/1chu8wc9zbNq...,https://api.spotify.com/v1/audio-analysis/1chu...,281227.0,4.0
3,3,God Only Knows - Mono / 1997 Remastered,6iGU74CwXuT4XVepjc9Emf,The Beach Boys,0.521,0.487,6.0,-7.525,0.0,0.0299,...,0.0579,0.483,117.072,audio_features,6iGU74CwXuT4XVepjc9Emf,spotify:track:6iGU74CwXuT4XVepjc9Emf,https://api.spotify.com/v1/tracks/6iGU74CwXuT4...,https://api.spotify.com/v1/audio-analysis/6iGU...,173040.0,4.0
4,4,Bridge Over Troubled Water,6l8EbYRtQMgKOyc1gcDHF9,Simon & Garfunkel,0.149,0.206,3.0,-13.888,1.0,0.0323,...,0.1150,0.264,79.764,audio_features,6l8EbYRtQMgKOyc1gcDHF9,spotify:track:6l8EbYRtQMgKOyc1gcDHF9,https://api.spotify.com/v1/tracks/6l8EbYRtQMgK...,https://api.spotify.com/v1/audio-analysis/6l8E...,293120.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,She Will Be Loved,7sapKrjDij2fpDVj0GxP66,Maroon 5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,996,Achy Breaky Heart,2EoIt9vdgFRNW03u5IvFsQ,Billy Ray Cyrus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,997,Reet Petite,3h4udS0WeWbsur3yfjvnm4,Jackie Wilson,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,998,All I Want for Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,Mariah Carey,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
fourth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")

In [25]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fourth_results = get_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")

In [26]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fourth_results
for r in fourth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist4_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [27]:
features4_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))

In [28]:
features4_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))

In [29]:
fifth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")


In [30]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fifth_results = get_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")

In [31]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fifth_results
for r in fifth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist5_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [32]:
features5_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1k1Bqnv2R0uJXQN4u6LKYt,745H5CctFr12Mo7cqa1BMH,3zBhihYUHBmGd2bcQIobrF,63xdwScd1Ai1GigAwQxE8y,1Omt5bfz1tZUCqd26HxbS0,3nFJbZCHP4d9vduKjJLdBL,49MHCPzvMLXhRjDantBMVH,3gsCAGsWr6pUm1Vy7CPPob,3l3xTXsUXeWlkPqzMs7mPD,7MF7QAodbGzNYav5ZfIhAY,3VZmChrnVW8JK6ano4gSED,0n2pjCIMKwHSXoYfEbYMfX,3M8FzayQWtkvOhqMn2V4T2,7jmHyHMEqm9MJWiMAneF05,5K09WxKdlkroDyEVyOSySy,1lx8ddGT5wCD6W2xmLeRKG,10vkYRKw6Jjr7try1ir50G,3cfnGXJ9bmiWvFqEO6ff8B,3IDc87lrpEHWFBASmgTgFV,0KOE1hat4SIer491XKk4Pa,6Ro2z4RtAUew9Kz2HZWZUi,1j2LuIf7mv15ZVug1Xy5qz,7IljDTNkQEyy77CcaRbiAj,1ckU1EhAO0Nr73QYw24SWJ,2yCAGizViD7HZXwNYxJToQ,5ZoDwIP1ntHwciLjydJ8X2,23IK8TF2MSiLnuEjaYPfVV,7JOyY9GA3P2Evx50oetDKb,1qfDfZAvrkSAFrAR2FTQDv,73GR50SgGV5hybgWDSFWtD,6lUXneXZ9jc3HSMHgd8oqh,5GvWrvLIqoHroq7YvO260M,02srSkeu2pzybuVr2B9TJm,5HrVgmO1govSOrPUehW1VV,3nsJGZ5RV2ZHkbKvD771gU,39Bd345OWEhRNyfayhp9gv,61sQYdFNS6sEBYCyr1q5gn,2ezqQeBiC72gwMJoO4w1hA,0ACACkoHUwgfgY5CxVIL4N,7gh2v4IHnxdiwSgA6xluhe,0J8A1ZVyJcR

SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1k1Bqnv2R0uJXQN4u6LKYt,745H5CctFr12Mo7cqa1BMH,3zBhihYUHBmGd2bcQIobrF,63xdwScd1Ai1GigAwQxE8y,1Omt5bfz1tZUCqd26HxbS0,3nFJbZCHP4d9vduKjJLdBL,49MHCPzvMLXhRjDantBMVH,3gsCAGsWr6pUm1Vy7CPPob,3l3xTXsUXeWlkPqzMs7mPD,7MF7QAodbGzNYav5ZfIhAY,3VZmChrnVW8JK6ano4gSED,0n2pjCIMKwHSXoYfEbYMfX,3M8FzayQWtkvOhqMn2V4T2,7jmHyHMEqm9MJWiMAneF05,5K09WxKdlkroDyEVyOSySy,1lx8ddGT5wCD6W2xmLeRKG,10vkYRKw6Jjr7try1ir50G,3cfnGXJ9bmiWvFqEO6ff8B,3IDc87lrpEHWFBASmgTgFV,0KOE1hat4SIer491XKk4Pa,6Ro2z4RtAUew9Kz2HZWZUi,1j2LuIf7mv15ZVug1Xy5qz,7IljDTNkQEyy77CcaRbiAj,1ckU1EhAO0Nr73QYw24SWJ,2yCAGizViD7HZXwNYxJToQ,5ZoDwIP1ntHwciLjydJ8X2,23IK8TF2MSiLnuEjaYPfVV,7JOyY9GA3P2Evx50oetDKb,1qfDfZAvrkSAFrAR2FTQDv,73GR50SgGV5hybgWDSFWtD,6lUXneXZ9jc3HSMHgd8oqh,5GvWrvLIqoHroq7YvO260M,02srSkeu2pzybuVr2B9TJm,5HrVgmO1govSOrPUehW1VV,3nsJGZ5RV2ZHkbKvD771gU,39Bd345OWEhRNyfayhp9gv,61sQYdFNS6sEBYCyr1q5gn,2ezqQeBiC72gwMJoO4w1hA,0ACACkoHUwgfgY5CxVIL4N,7gh2v4IHnx

In [ ]:
fifth_playlist = pd.concat([playlist5_info, features5_info], axis=1)
fifth_playlist.reset_index()

In [ ]:
sixth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")


In [ ]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
sixth_results = get_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")

In [ ]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = sixth_results
for r in sixth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist6_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [ ]:
features6_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features6_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features6_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:]))
features6 = pd.concat([features6_info, features6_info2, features6_info3])
features6.reset_index(drop=True, inplace=True)

In [ ]:
sixth_playlist = pd.concat([playlist6_info, features6], axis=1)
sixth_playlist.reset_index()

In [ ]:

seventh_playlist = sp.user_playlist_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")

In [ ]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
seventh_results = get_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")

In [ ]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = seventh_results
for r in seventh_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist7_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [ ]:
features7_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features7_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features7_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features7_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features7_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features7_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:]))
features7 = pd.concat([features7_info, features7_info2, features7_info3, features7_info4, features7_info5, features7_info6])
features7.reset_index(drop=True, inplace=True)

In [ ]:
seventh_playlist = pd.concat([playlist7_info, features7], axis=1)
seventh_playlist.reset_index()

In [ ]:
eighth_playlist = sp.user_playlist_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")

In [ ]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
eighth_results = get_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")

In [ ]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = eighth_results
for r in eighth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist8_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [ ]:
features8_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features8_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features8_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features8_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features8_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features8_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features8_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features8_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features8_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features8_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features8_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features8 = pd.concat([features8_info, features8_info2, features8_info3, features8_info4, features8_info5, features8_info6, features8_info7, features8_info8, features8_info9, features8_info10, features8_info11])
features8.reset_index(drop=True, inplace=True)

In [ ]:
eighth_playlist = pd.concat([playlist8_info, features8], axis=1)
eighth_playlist.reset_index()

In [ ]:

ninth_playlist = sp.user_playlist_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")

In [ ]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
ninth_results = get_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")

In [ ]:

song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = ninth_results
for r in ninth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist9_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [ ]:
features9_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features9_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features9_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features9_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features9_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features9_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features9_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features9_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features9_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features9_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features9_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features9 = pd.concat([features9_info, features9_info2, features9_info3, features9_info4, features9_info5, features9_info6, features9_info7, features9_info8, features9_info9, features9_info10, features9_info11])
features9.reset_index(drop=True, inplace=True)

In [ ]:
ninth_playlist = pd.concat([playlist9_info, features9], axis=1)
ninth_playlist.reset_index()

In [ ]:
huge_playlist = pd.concat([first_playlist, second_playlist, third_playlist, fourth_playlist, 
                           sixth_playlist, seventh_playlist, eighth_playlist, 
                           ninth_playlist])
huge_playlist.reset_index(drop=True, inplace=True)

In [ ]:
# count duplicated values and dropping them
huge_playlist.duplicated(subset='song_name', keep='first').sum()
huge_playlist = huge_playlist.drop_duplicates()
huge_playlist.reset_index(drop=True, inplace=True)
huge_playlist = huge_playlist.dropna()
huge_playlist.reset_index(drop=True, inplace=True)

In [ ]:
# even MORE packages!
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly 
import plotly.graph_objs as go
from sklearn.cluster import KMeans 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler 

In [ ]:
# copying the playlist just in case
hugeplaylist = huge_playlist.copy()

In [ ]:
# Song as index! yay!
hugeplaylist.index = hugeplaylist.iloc[:,0]
# pick first column and turn it to index [:,_] - location

In [ ]:
#keep rest of columns
hugeplaylist = hugeplaylist.iloc[:,1:]

In [ ]:
# remove the header name
hugeplaylist.rename_axis(None,inplace=True)

In [ ]:

hugeplaylist_scaled = StandardScaler().fit_transform(hugeplaylist)

In [ ]:
hugeplaylist_scaled_df = pd.DataFrame(hugeplaylist_scaled,columns = ['danceability', 'energy', 
                                                                     'key', 'loudness', 
                                                                     'speechiness', 'acousticness', 
                                                                     'instrumentalness', 'liveness', 
                                                                     'valence', 'tempo'])

In [ ]:
kmeans = KMeans(n_clusters = 52)

In [ ]:
cluster = kmeans.fit(hugeplaylist_scaled_df)

In [ ]:
#bring cluster into data frame
hugeplaylist_c = hugeplaylist.copy()
hugeplaylist_c['cluster'] = cluster.labels_

In [ ]:

# creating the cluster thingy
from IPython.core.display import display
from IPython.display import IFrame


def song_cluster(song):
    song = sp.search(q = song, limit = 1)
    song_id = song['tracks']
    song_id = song_id['items'][0]['id']
    song_features = sp.audio_features(song_id)
    song_data = pd.DataFrame(song_features, index= [0])
   # droping the numeric columns i didn't use
    song_data.drop(['mode', 'duration_ms', 'time_signature'], axis = 1, inplace = True)  
    song_num = song_data._get_numeric_data()
    song_scaled = StandardScaler().fit_transform(song_num)
    cluster = kmeans.predict(song_scaled)
    song_cluster = huge_playlist.loc[huge_playlist['cluster'] == int(cluster)]
    song_cluster.reset_index(drop=True, inplace=True) 
    song_name_id = random.choice(song_cluster["id"])
    songname = sp.track(song_name_id)['name']
    artist = sp.track(song_name_id)['artists'][0]['name']
    def recommend(song_name_id):
        display(IFrame(src=f"https://open.spotify.com/embed/track/{song_name_id}",
                        width="320",
                        height="80",
                        frameborder="0",
                        allowtransparency="true",
                        allow="encrypted-media",))
    return print("\033[1m" + 'That song is not hot right now. But listen to ' + songname + ' by ' + artist + ". I think you'll like it!."

In [ ]:
# new suggester attempt

def second_suggester():
    song = str(input( "\033[1m" + """
    Welcome to the awesome song recommender!
    What song do you have in mind?: 
    """).lower().replace(" ", ''))
    # check hot 100
    check = billboard_top100[billboard_top100['song'].str.lower().str.replace(" ","").str.contains(song)]
    # get index
    index = check.index.tolist()
    if len(check) == 0:
        song_cluster(song)
        song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
        if song2 == 'y':
            print("\033[1m" + 'Awesome! Here, have another go:')
            second_suggester()
        elif song2 != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
            
        elif song2 == 'n':
            print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
            return
    else:
        answer1 = input("\033[1m" + "Are you thinking about '" + billboard_top100.song[index].values[0] + "' by '" + billboard_top100.artist[index].values[0] + "'? [y/n]: ")
        #make a song suggestion
        if answer1.lower() == 'y':
            suggestion = billboard_top100.sample().index.tolist()
            print("\033[1m" + "That's a hot song! Listen to '" + billboard_top100['song'][suggestion].item() + "' by '" + billboard_top100['artist'][suggestion].item() + "' I think you'll like it.")
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            if song2 == 'y':
                print("\033[1m" + 'Awesome! Here, have another go:')
                second_suggester()
            elif song2 != 'n':
                print("\033[1m" + "Oh, come on, that's not an answer")
                song2
            elif song2 == 'n':
                print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
                return
        elif answer1.lower() != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            second_suggester()
        elif answer1.lower() == 'n':
            song_cluster(song)
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            if song2 == 'y':
                print("\033[1m" + 'Awesome! Here, have another go:')
                second_suggester()
            elif song2 != 'n':
                print("\033[1m" + "Oh, come on, that's not an answer")
                song2
            elif song2 == 'n':
                print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
                return
        elif song2 != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
            song2
        elif song2 == 'n':
            print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
            return
second_suggester()